In [5]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import json
from shapely.geometry import Point
from folium.plugins import MarkerCluster, FeatureGroupSubGroup

In [8]:
import geopandas as gpd

# 파일 경로 설정
path = './'
tbin = 'recommended_trash_bins.csv'
lb = 'legal_boundary.geojson'

# GeoJSON 파일 로드
trash_bins = gpd.read_file(path + tbin)  # 쓰레기통 위치
legal_boundary = gpd.read_file(path + lb)  # 법정 경계

df = pd.DataFrame(trash_bins)

# GeoDataFrame으로 변환
gdf = gpd.GeoDataFrame(
    df,
    geometry=[Point(xy) for xy in zip(df['longitude'], df['latitude'])],
    crs="EPSG:4326"  # WGS84 좌표계
)
gdf.head()
trash_bins = gdf

In [9]:


# 좌표계 통일
trash_bins = trash_bins.to_crs(legal_boundary.crs)

# 공간 조인: 쓰레기통 좌표가 구 경계 안에 포함되는지 확인
trash_bins_with_districts = gpd.sjoin(trash_bins, legal_boundary, how="left", predicate="within")

# 결과 저장 (옵션)
# trash_bins_with_districts.to_file(path + 'trash_bins_with_districts.geojson', driver='GeoJSON')



In [10]:
# trash_bins_with_districts.dropna(axis = 0,inplace = True )
trash_bins_with_districts.isna().sum()

latitude       0
longitude      0
score          0
geometry       0
index_right    0
SIG_CD         0
SIG_KOR_NM     0
dtype: int64

In [13]:
# 결과 확인
# trash_bins_with_districts = trash_bins_with_districts.drop(columns=['index_right','SIG_CD'])
display(trash_bins_with_districts.head())
trash_bins_with_districts.to_file('rc_trash_bins.geojson', driver='GeoJSON')

,latitude,longitude,score,geometry,SIG_KOR_NM
0,37.466698724120896,126.89050883155603,0.602538773953227,POINT (126.89051 37.4667),금천구
1,37.5020838257097,126.90471572236515,0.5787335334598886,POINT (126.90472 37.50208),영등포구
2,37.529939903626136,127.07459046142202,0.5745836940342952,POINT (127.07459 37.52994),광진구
3,37.47247653659869,127.04498066380205,0.5591038360167819,POINT (127.04498 37.47248),서초구
4,37.491489545131415,127.08167492828633,0.5461847875097152,POINT (127.08167 37.49149),강남구


In [21]:
# # import folium
# # from folium.plugins import MarkerCluster

# # 이것 사용
# # 중심 좌표 설정 (예: 서울시청 좌표)
# center_lat, center_lon = 37.5665, 126.9780

# # 폴리움 지도 생성
# m = folium.Map(location=[center_lat, center_lon], zoom_start=11)

# # 1. 서울시 통합 경계 및 쓰레기통 레이어
# seoul_layer = folium.FeatureGroup(name="서울시 통합 경계 및 쓰레기통", show=True)

# # 서울시 전체 경계 추가
# folium.GeoJson(
#     legal_boundary,
#     tooltip="서울시 경계"
# ).add_to(seoul_layer)

# # 모든 쓰레기통 마커 추가
# all_marker_cluster = MarkerCluster().add_to(seoul_layer)

# for _, row in trash_bins_with_districts.iterrows():
#     folium.Marker(
#         location=[row['geometry'].y, row['geometry'].x],
#         tooltip=f"구: {row['SIG_KOR_NM']}"
#     ).add_to(all_marker_cluster)

# seoul_layer.add_to(m)

# # 2. 구별 경계 및 쓰레기통 통합 레이어
# for district_name in trash_bins_with_districts['SIG_KOR_NM'].unique():
#     # FeatureGroup 생성
#     district_layer = folium.FeatureGroup(name=f"{district_name}", show=False)
    
#     # 구별 경계 추가
#     district_boundary = legal_boundary[legal_boundary['SIG_KOR_NM'] == district_name]
#     folium.GeoJson(
#         district_boundary,
#         tooltip=district_name  # 경계 툴팁
#     ).add_to(district_layer)
    
#     # 해당 구의 쓰레기통 데이터 필터링
#     district_trash_bins = trash_bins_with_districts[trash_bins_with_districts['SIG_KOR_NM'] == district_name]
#     marker_cluster = MarkerCluster().add_to(district_layer)
    
#     for _, row in district_trash_bins.iterrows():
#         folium.Marker(
#             location=[row['geometry'].y, row['geometry'].x],
#             tooltip=f"구: {district_name}"
#         ).add_to(marker_cluster)
    
#     district_layer.add_to(m)

# # 레이어 컨트롤 추가
# folium.LayerControl().add_to(m)

# # 지도 저장 (옵션)
# m.save('trash_bins_map_with_seoul_layer.html')

# # 결과 지도 출력
# # m
